# ДЗ по "Сбор и разметка данных (семинары)"
## Семинар 3. Системы управления базами данных MongoDB и Кликхаус в Python
* Домашнее задание:
1.	Установите MongoDB на локальной машине, а также зарегистрируйтесь в онлайн-сервисе. https://www.mongodb.com/ https://www.mongodb.com/products/compass
2.	Загрузите данные который вы получили на предыдущем уроке путем скрейпинга сайта с помощью Buautiful Soup в MongoDB и создайте базу данных и коллекции для их хранения.
3.	Поэкспериментируйте с различными методами запросов.
4.	Зарегистрируйтесь в ClickHouse.
5.	Загрузите данные в ClickHouse и создайте таблицу для их хранения

## Установка MongoDB Community Server на Ubuntu 22.04
У меня была виртульная машина (ВМ) с ОС Linux Ubuntu 22.04 cо статическим IP (см. [Семинар 1. Установка ОС Ubuntu в виртуальной машине. Работа в SSH-клиенте](https://github.com/uc20100/Linux/blob/main/seminar_1.md), [Семинар 5. Настройка сети в Linux. Работа с IPtables](https://github.com/uc20100/Linux/blob/main/seminar_5.md))  
Далее используя материал ссылок [Установка MongoDB в Ubuntu 20.04](https://www.digitalocean.com/community/tutorials/how-to-install-mongodb-on-ubuntu-20-04-ru) и [MongoDB Install Fails on Ubuntu 22.04](https://askubuntu.com/questions/1403619/mongodb-install-fails-on-ubuntu-22-04-depends-on-libssl1-1-but-it-is-not-insta) я составил следующий текст, который копируется в терминал Ubuntu и нажимается `Enter`:
```shell
echo "deb http://security.ubuntu.com/ubuntu focal-security main" | sudo tee /etc/apt/sources.list.d/focal-security.list
sudo apt-get update
sudo apt-get install libssl1.1

curl -fsSL https://pgp.mongodb.com/server-4.4.asc | sudo apt-key add -
apt-key list
echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/4.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.4.list
sudo apt update
sudo apt install mongodb-org

sudo systemctl start mongod.service
sudo systemctl status mongod
sudo systemctl enable mongod

mongo --eval 'db.runCommand({ connectionStatus: 1 })'
```

Далее нужно изменить файл конфигурации MongoDB, введем следующею команду в терминале:
```shell
sudo nano /etc/mongod.conf
```
И изменить строчку, добавив IP Ubuntu (в моем случае у меня IP 192.168.1.25) 
```text
bindIp: 127.0.0.1, 192.168.1.25
```

Перезапускаем сервер 
```shell
sudo systemctl restart mongo
```
Всё, готово. Теперь к нашему серверу можно подключаться с помощью MongoDB Compass указав вместо `localhost` IP Ubuntu.  
Дополнительно - [Установка MongoDB в Ubuntu 22.04](https://gohost.kz/blog/vps/ustanovka-mongodb-v-ubuntu-22-04/)

## Поработаем с MongoDB Community Server в Python

In [1]:
# # Устанавливаем пакеты, если они не установлены
# !pip install pymongo
# !pip install clickhouse-driver
# !pip install pandas

In [2]:
import json
from pymongo import MongoClient

In [3]:
# Подключение к серверу MongoDB
client = MongoClient('mongodb://192.168.1.25:27017/')

In [4]:
# Выбор базы данных и коллекции
db = client['books']
collection = db['books_toscrape_com']

In [5]:
# Чтение файла JSON
with open('result.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [6]:
# # Вставка книг в коллекцию MongoDB
for item in data:
    collection.insert_one(item)

### Работаем с базой данных
[Конспект лекции](https://gbcdn.mrgcdn.ru/uploads/asset/5621409/attachment/47e385b893efafe9255cb3ed1d090c44.pdf) 

In [7]:
# вывод первой записи в коллекции
all_docs = collection.find()
first_doc = all_docs[0]
print(first_doc)

{'_id': ObjectId('66025d5350918972f5963ddd'), 'category': 'Travel', 'name': "It's Only the Himalayas", 'price': 45.17, 'available': 19, 'description': '“Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and wa “Wherever you go, whatever you do, just . . . don’t do anything stupid.” —My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as "stupid." She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and watched as her friend was attacked by a monkey in Indonesia.But interspersed in those slightly more

In [8]:
# Вывод объекта JSON
pretty_json = json.dumps(first_doc, indent=4, default=str)
print(pretty_json)

{
    "_id": "66025d5350918972f5963ddd",
    "category": "Travel",
    "name": "It's Only the Himalayas",
    "price": 45.17,
    "available": 19,
    "description": "\u201cWherever you go, whatever you do, just . . . don\u2019t do anything stupid.\u201d \u2014My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as \"stupid.\" She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and wa \u201cWherever you go, whatever you do, just . . . don\u2019t do anything stupid.\u201d \u2014My MotherDuring her yearlong adventure backpacking from South Africa to Singapore, S. Bedford definitely did a few things her mother might classify as \"stupid.\" She swam with great white sharks in South Africa, ran from lions in Zimbabwe, climbed a Himalayan mountain without training in Nepal, and watched as her friend was attacked by a 

In [9]:
# Получение количества документов в коллекции с помощью функции count_documents()
count = collection.count_documents({})
print(f'Число записей в базе данных: {count}')

Число записей в базе данных: 1000


In [10]:
# фильтрация документов по критериям
query = {'category': 'Travel'}
print(f"Количество документов c категорией 'Travel': {collection.count_documents(query)}")

Количество документов c категорией 'Travel': 11


In [11]:
# Использование проекции
query = {'category': 'Travel'}
projection = {"name": 1, "price": 1, "available": 1, "_id": 0}
proj_docs = collection.find(query, projection)
for doc in proj_docs:
    print(doc)

{'name': "It's Only the Himalayas", 'price': 45.17, 'available': 19}
{'name': 'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond', 'price': 49.43, 'available': 15}
{'name': 'See America: A Celebration of Our National Parks & Treasured Sites', 'price': 48.87, 'available': 14}
{'name': 'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'price': 36.94, 'available': 8}
{'name': 'Under the Tuscan Sun', 'price': 37.33, 'available': 7}
{'name': 'A Summer In Europe', 'price': 44.34, 'available': 7}
{'name': 'The Great Railway Bazaar', 'price': 30.54, 'available': 6}
{'name': 'A Year in Provence (Provence #1)', 'price': 56.88, 'available': 6}
{'name': 'The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)', 'price': 23.21, 'available': 3}
{'name': 'Neither Here nor There: Travels in Europe', 'price': 38.95, 'available': 3}
{'name': '1,000 Places to See Before You Die', 'price': 26.08, 'available': 1}


In [12]:
# Использование оператора $lt и $gte
AVAILABLE_1 = 2
AVAILABLE_2 = 20
query = {"available": {"$lt": AVAILABLE_1}}
print(f"Количество документов c категорией available < {AVAILABLE_1}: {collection.count_documents(query)}")
query = {"available": {"$gte": AVAILABLE_2}}
print(f"Количество документов c категорией available >= {AVAILABLE_2}: {collection.count_documents(query)}")

Количество документов c категорией available < 2: 98
Количество документов c категорией available >= 20: 5


In [13]:
# Использование оператора $regex
WORD = "America"
query = {"name": {"$regex": WORD, "$options": "i"}}
print(f"Количество документов, содержащих '{WORD}': {collection.count_documents(query)}")

Количество документов, содержащих 'America': 25


In [14]:
# Использование оператора $in
query = {"category": {"$in": ["Travel", "Romance", "Science Fiction"]}}
print(f"Количество документов в категории 'category': {collection.count_documents(query)}")

Количество документов в категории 'category': 62


In [15]:
# Использование оператора $all
query = {"category": {"$all": ["Mystery"]}}
print(f"Количество документов в категории 'category': {collection.count_documents(query)}")

Количество документов в категории 'category': 32


In [16]:
# Использование оператора $ne
query = {"category" : {"$ne": "Mystery"}}
print(f"Количество документов в категории 'category': {collection.count_documents(query)}")

Количество документов в категории 'category': 968


## Устанавливаем ClickHouse на Ubuntu 22.04
Эксперементальным путём установил на Ubuntu, ClickHouse. Ниже приводится текст, который нужно скопировать и втавить в терминал Ubuntu и нажать `Enter`. Когда при установе ClickHouse Server будет запрашиваться пароль - пароль не вводить, нажать `Enter`  
```shell
sudo apt update && sudo apt upgrade -
sudo apt install dirmngr ca-certificates software-properties-common apt-transport-https curl
GNUPGHOME=$(mktemp -d)
sudo GNUPGHOME="$GNUPGHOME" gpg --no-default-keyring --keyring /usr/share/keyrings/clickhouse-keyring.gpg --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 8919F6BD2B48D754
sudo rm -rf "$GNUPGHOME"
sudo chmod +r /usr/share/keyrings/clickhouse-keyring.gpg
echo "deb [arch=amd64 signed-by=/usr/share/keyrings/clickhouse-keyring.gpg] https://packages.clickhouse.com/deb stable main" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
sudo apt update
sudo apt install clickhouse-server clickhouse-client
sudo systemctl start clickhouse-server
sudo systemctl enable clickhouse-server
sudo systemctl status clickhouse-server
```
Надо разрешить нашему приложению, коннектится к серверу ClickHouse, для этого отредактируем файл конфигурации.
```shell
sudo nano /etc/clickhouse-server/config.xml
```
Раскомментируем строчку, чтобы могли подключаться все (это не очень хорошо, но для экспериментов пойдет)
```text
<listen_host>0.0.0.0</listen_host>
```
Перезапустим сервер
```shell
sudo systemctl restart clickhouse-server
sudo systemctl status clickhouse-server
```
Все должно заработать.  

* Дополнительная инфа:  
[Как установить ClickHouse на Ubuntu 22.04 LTS Linux](https://www.how2shout.com/linux/how-to-install-clickhouse-on-ubuntu-22-04-lts-linux/)  
[Установка и использование ClickHouse на Linux](https://www.dmosk.ru/miniinstruktions.php?mini=clickhouse-linux)

## Поработаем с ClickHouse в Python

In [17]:
import pandas as pd
# Настройка формата вывода чисел float
pd.set_option('display.float_format', '{:.2f}'.format)
from clickhouse_driver import Client

In [18]:
# Подключение к серверу ClickHouse
client = Client('192.168.1.25')

In [19]:
# Создание базы данных (если она не существует)
client.execute('CREATE DATABASE IF NOT EXISTS books')

[]

In [20]:
# Создание таблицы
# Создание основной таблицы 'books_toscrape_com'
client.execute('''
CREATE TABLE IF NOT EXISTS books.books_toscrape_com (
    id String,
    category String,
    name String,
    price Float32,
    available Int16,
    description String
) ENGINE = MergeTree()
ORDER BY id
''')

[]

In [21]:
# Вставка данных в таблицу
for item in data:

    # Вставка данных о книгах
    client.execute("""
    INSERT INTO books.books_toscrape_com (
        id, category, name,
        price, available, description
    ) VALUES""",
    [(item['name'],
      item['category'] or "",
      item['name'] or "",
      item['price'] or 0,
      item['available'] or 0,
      item['description'] or "")])

In [22]:
# Проверка успешности вставки
result = client.execute("SELECT * FROM books.books_toscrape_com")
print("Вставленная запись:", result[0])

Вставленная запись: ('Amid the Chaos', 'Cultural', 'Amid the Chaos', 36.58000183105469, 15, 'Some people call Eritrea the “North Korea of Africa.” But to two friends Chenkelo and Misghe, it is home. In the picturesque capital of Asmara, these two educated young men are forced to choose between poverty and hustle as they hide from a national service conscription that would send them to the front lines. Charismatic Misghe is a charmer, a philosopher, and a loving so Some people call Eritrea the “North Korea of Africa.” But to two friends Chenkelo and Misghe, it is home. In the picturesque capital of Asmara, these two educated young men are forced to choose between poverty and hustle as they hide from a national service conscription that would send them to the front lines. Charismatic Misghe is a charmer, a philosopher, and a loving son who is capable of having any woman in the city. But he’s about to flee his oppressive homeland—putting his longtime friendship with Chenkelo at stake. Che

In [23]:
# 1. Выполнение базового запроса для получения всех записей из таблицы 'books_toscrape_com'
records = client.execute('SELECT * FROM books.books_toscrape_com')
df_records = pd.DataFrame(records, columns=['id', 'category', 'name', 'price', 'available', 'description'])
df_records.head()

,id,category,name,price,available,description
0,Amid the Chaos,Cultural,Amid the Chaos,36.58,15,Some people call Eritrea the “North Korea of A...
1,Dark Notes,Erotica,Dark Notes,19.19,15,They call me a slut. Maybe I am.Sometimes I do...
2,Equal Is Unfair: America's Misguided Fight Aga...,Politics,Equal Is Unfair: America's Misguided Fight Aga...,56.86,12,We’ve all heard that the American Dream is van...
3,Libertarianism for Beginners,Politics,Libertarianism for Beginners,51.33,19,Libertarianism isn't about winning elections; ...
4,The Long Shadow of Small Ghosts: Murder and Me...,Crime,The Long Shadow of Small Ghosts: Murder and Me...,10.97,15,In Cold Blood meets Adrian Nicole LeBlanc’s Ra...


In [24]:
# 2. Фильтрация записей на основе критериев
# 2.1. Равенство: Получение записей с category, равным 'Travel'
travel_records = client.execute("SELECT * FROM books.books_toscrape_com WHERE category = 'Travel'")
df_travel_records = pd.DataFrame(travel_records, columns=df_records.columns)
df_travel_records.head(20)

,id,category,name,price,available,description
0,"1,000 Places to See Before You Die",Travel,"1,000 Places to See Before You Die",26.08,1,"Around the World, continent by continent, here..."
1,A Summer In Europe,Travel,A Summer In Europe,44.34,7,"On her thirtieth birthday, Gwendolyn Reese rec..."
2,A Year in Provence (Provence #1),Travel,A Year in Provence (Provence #1),56.88,6,National BestsellerIn this witty and warm-hear...
3,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,15,Acclaimed travel writer Rick Antonson sets his...
4,It's Only the Himalayas,Travel,It's Only the Himalayas,45.17,19,"“Wherever you go, whatever you do, just . . . ..."
5,Neither Here nor There: Travels in Europe,Travel,Neither Here nor There: Travels in Europe,38.95,3,"Bill Bryson's first travel book, The Lost Cont..."
6,See America: A Celebration of Our National Par...,Travel,See America: A Celebration of Our National Par...,48.87,14,To coincide with the 2016 centennial anniversa...
7,The Great Railway Bazaar,Travel,The Great Railway Bazaar,30.54,6,"First published more than thirty years ago, Pa..."
8,The Road to Little Dribbling: Adventures of an...,Travel,The Road to Little Dribbling: Adventures of an...,23.21,3,The hilarious and loving sequel to a hilarious...
9,Under the Tuscan Sun,Travel,Under the Tuscan Sun,37.33,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...


In [25]:
# 2.3. Диапазон: Выборка записей по полю available в диапазоне от 18 до 20
AVAILABLE_1 = 18
AVAILABLE_2 = 20
range_records = client.execute(f"SELECT * FROM books.books_toscrape_com WHERE available BETWEEN {AVAILABLE_1} AND {AVAILABLE_2}")
df_range_records = pd.DataFrame(range_records, columns=df_records.columns)
df_range_records

,id,category,name,price,available,description
0,Libertarianism for Beginners,Politics,Libertarianism for Beginners,51.33,19,Libertarianism isn't about winning elections; ...
1,Aladdin and His Wonderful Lamp,Default,Aladdin and His Wonderful Lamp,53.13,19,Planning on seeing Aladdin the Disney musical?...
2,America's Cradle of Quarterbacks: Western Penn...,Default,America's Cradle of Quarterbacks: Western Penn...,22.50,19,"Any fan of the Baltimore Colts of my era, any ..."
3,Behind Closed Doors,Thriller,Behind Closed Doors,52.22,18,The 2016 debut bloggers can't stop raving abou...
4,Birdsong: A Story in Pictures,Childrens,Birdsong: A Story in Pictures,54.64,19,Bring the thrilling story of one red bird to l...
5,Black Dust,Romance,Black Dust,34.53,19,"No matter how busy he keeps himself, successfu..."
6,Chase Me (Paris Nights #2),Romance,Chase Me (Paris Nights #2),25.27,19,"A Michelin two-star chef at twenty-eight, Viol..."
7,Foolproof Preserving: A Guide to Small Batch J...,Food and Drink,Foolproof Preserving: A Guide to Small Batch J...,30.52,19,Canning perfected the America’s Test Kitchen w...
8,How Music Works,Music,How Music Works,37.32,19,How Music Works is David Byrne’s remarkable an...
9,In Her Wake,Thriller,In Her Wake,12.84,19,A perfect life … until she discovered it wasn’...


In [26]:
# 3. Сортировка записей на основе одного или нескольких полей
# 3.1. Сортировка записей по наличию порядке убывания
sorted_records = client.execute("SELECT * FROM books.books_toscrape_com ORDER BY available DESC")
df_sorted_records = pd.DataFrame(sorted_records, columns=df_records.columns)
df_sorted_records.head(10)

,id,category,name,price,available,description
0,A Light in the Attic,Poetry,A Light in the Attic,51.77,22,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,Historical Fiction,Tipping the Velvet,53.74,20,"""Erotic and absorbing...Written with starling ..."
2,Soumission,Fiction,Soumission,50.10,20,"Dans une France assez proche de la nôtre, un h..."
3,Sharp Objects,Mystery,Sharp Objects,47.82,20,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,History,Sapiens: A Brief History of Humankind,54.23,20,From a renowned historian comes a groundbreaki...
5,Libertarianism for Beginners,Politics,Libertarianism for Beginners,51.33,19,Libertarianism isn't about winning elections; ...
6,The Requiem Red,Young Adult,The Requiem Red,22.65,19,Patient Twenty-nine.A monster roams the halls ...
7,The Dirty Little Secrets of Getting Your Dream...,Business,The Dirty Little Secrets of Getting Your Dream...,33.34,19,Drawing on his extensive experience evaluating...
8,The Coming Woman: A Novel Based on the Life of...,Default,The Coming Woman: A Novel Based on the Life of...,17.93,19,"""If you have a heart, if you have a soul, Kare..."
9,The Boys in the Boat: Nine Americans and Their...,Default,The Boys in the Boat: Nine Americans and Their...,22.60,19,For readers of Laura Hillenbrand's Seabiscuit ...


In [27]:
# 3.2. Сортировка записей по наличию в порядке возрастания и по цене в порядке убывания
multi_sorted_records = client.execute("SELECT * FROM books.books_toscrape_com ORDER BY available ASC, price DESC")
df_multi_sorted_records = pd.DataFrame(multi_sorted_records, columns=df_records.columns)
df_multi_sorted_records.head(1000)

,id,category,name,price,available,description
0,Listen to Me (Fusion #1),Romance,Listen to Me (Fusion #1),58.99,1,In New York Times and USA Today bestselling au...
1,Myriad (Prentor #1),Fantasy,Myriad (Prentor #1),58.75,1,Alternate Cover Edition: 14783849642016 Revise...
2,Shameless,New Adult,Shameless,58.35,1,***USA TODAY BESTSELLER***Brady…What the hell ...
3,Travels with Charley: In Search of America,Nonfiction,Travels with Charley: In Search of America,57.82,1,An intimate journey across and in search of Am...
4,The No. 1 Ladies' Detective Agency (No. 1 Ladi...,Mystery,The No. 1 Ladies' Detective Agency (No. 1 Ladi...,57.70,1,The No.1 ladies' detective agency consists of ...
...,...,...,...,...,...,...
995,Sapiens: A Brief History of Humankind,History,Sapiens: A Brief History of Humankind,54.23,20,From a renowned historian comes a groundbreaki...
996,Tipping the Velvet,Historical Fiction,Tipping the Velvet,53.74,20,"""Erotic and absorbing...Written with starling ..."
997,Soumission,Fiction,Soumission,50.10,20,"Dans une France assez proche de la nôtre, un h..."
998,Sharp Objects,Mystery,Sharp Objects,47.82,20,"WICKED above her hipbone, GIRL across her hear..."


In [28]:
# 4. Агрегировать записи с помощью таких функций, как count, sum и avg.
# 4.1. Подсчет общего количества записей
count_records = client.execute("SELECT AVG(price) FROM books.books_toscrape_com")
print("Средняя цена книги:", round(count_records[0][0], 2))

Средняя цена книги: 35.07


In [29]:
# 4.2. Подсчет количества книг в каждой категории
category_count_records = client.execute("SELECT category, COUNT(*) FROM books.books_toscrape_com GROUP BY category")
df_category_count_records = pd.DataFrame(category_count_records, columns=['category', 'count'])
df_category_count_records

,category,count
0,Romance,35
1,Humor,10
2,Mystery,32
3,Suspense,1
4,Default,152
5,Health,4
6,Erotica,1
7,Poetry,19
8,Cultural,1
9,Horror,17


In [30]:
# 4.3. Рассчитайте среднюю цену книги в категории Travel
avg_crash_date = client.execute("SELECT AVG(price) FROM books.books_toscrape_com WHERE category = 'Travel'")
print("Средняя цена книги в категории 'Travel':", round(avg_crash_date[0][0], 2))

Средняя цена книги в категории 'Travel': 39.79


## Удаляем базы данных на серверах ClickHouse и MongoDB

In [31]:
# Удалим базу данных на сервере ClickHouse
client.execute('drop database books')

# Удалим базу данных на сервере MongoDB
db.drop_collection('books_toscrape_com')

{'nIndexesWas': 1, 'ns': 'books.books_toscrape_com', 'ok': 1.0}

## Вывод:
Работа с БД в MongoDB очень простая, порадовал клиент MongoDB Compass, для визуальной работы с базой данных (БД). Эта БД позволяет пользователю делать много вольностей, не заморачиваясь с архитектурой БД. 

ClickHouse очень похож на MySQL, но есть свои особенности. Очень быстрая по отзывам ([ClickHouse: как обрабатывать big data в 800 раз быстрее]( https://tproger.ru/articles/clickhouse-kak-obrabatyvat-big-data-v-800-raz-bystree), [Тестирование производительности аналитических запросов в PostgreSQL, ClickHouse и clickhousedb_fdw (PostgreSQL)](https://habr.com/ru/articles/511992/))  

Также попрактиковался в установке MongoDB и ClickHouse на Ubuntu 22.04. Установка очень упрощенная (чтобы работало, без всяких настроек), это можно сказать baseline, далее можно поэкспериментировать с установкой в контейнеры Docker с помощью Docker Compose.